In [67]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import json


def get_hotel_data(hotel_name):
    # Base search URL
    url = f'https://www.dineout.co.in/surat-restaurants?search_str={hotel_name}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }

    # Fetch the search page
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Get the first hotel object
    first_object = soup.find_all('div', class_='restnt-card-wrap-new')
    if not first_object:
        print("No hotels found!")
        return None

    first_object = first_object[0]

    # Extract the "data-link" value
    data_link = first_object.find('div', class_='img cursor').get('data-link')
    if not data_link:
        print("No data-link found in the first object!")
        return None

    # Construct full hotel URL
    hotel_url = "https://www.dineout.co.in" + data_link

    # -------------------------------- Selenium for Dynamic Content --------------------------------
    # Initialize Selenium WebDriver
    service = Service(r'/content/chromedriver.exe')  # Update the path if needed
    driver = webdriver.Chrome(service=service)

    try:
        # Open the hotel page
        driver.get(hotel_url)

        # Wait for the menu gallery to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "demo-gallery"))
        )

        # Extract page source after rendering
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract the <ul> element containing the menu
        menu_gallery = soup.find('ul', class_='d-flex demo-gallery')
        if not menu_gallery:
            print("No menu gallery found!")
            return None

        # Extract all menu links
        menu_links = []
        for link in menu_gallery.find_all('a'):
            href = link.get('href')
            if href:
                menu_links.append(href)

        # Return collected data as a dictionary
        result = {
            "hotel_name": hotel_name,
            "hotel_url": hotel_url,
            "menu_links": menu_links
        }
        return result

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    finally:
        # Quit Selenium WebDriver
        driver.quit()

In [68]:
hotel_name = "Zhingalala"#input("Enter the hotel name: ")
get_hotel_data(hotel_name)

WebDriverException: Message: 'chromedriver.exe' executable may have wrong permissions.
